<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png" width="40px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">3a - Workshop (AI Extensions) - Simple Decision Prompts</span>
</div>

Decision Intelligence concepts applied in this module:  
* Listing generic decision-making tools trained-into Generative AI LLMs   
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundamental building blocks for eliciting effective responses from AI models. This module demonstrates how to use common prompt engineering techniques within Microsoft AI Extensions. If you’ve used ChatGPT or Microsoft Copilot, you’re already familiar with prompting: given an instruction, a language model predicts the most likely and relevant response. With Microsoft AI Extensions, you can build applications, services, and APIs that execute these prompts quickly and effectively. 

For more information on using prompts with Microsoft Extensions AI, visit: https://learn.microsoft.com/en-us/dotnet/ai/quickstarts/prompt-model?pivots=azure-openai 

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal is to maximize the quality and clarity of the AI’s output, often by using specific wording, added context, or concrete examples within the prompt. 

> 📝 **Note:** Future modules will evolve the prompts to include **Context Engineering**, which is a more broader technique that includes setting up an information architecture (memory, exeternal data, system prompts, MCP Tools etc.) for making better decisions. 

By combining Decision Intelligence with Prompt Engineering/Context Engineering, you can create **“Decision Intelligence powered by Generative AI”**. This approach leverages Generative AI models to apply decision-making pricingples, by reasoning through complex decision scenarios, gathering intelligence, recommending decisions, and communicating results more effectively. 

-----
### Step 1 - Initialize Configuration Builder & Build the AI Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the AI orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 10.0.1"
#r "nuget: Microsoft.Extensions.Configuration.Json, 10.0.1"
#r "nuget: System.Text.Json, 10.0.1"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;
using System;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connection info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 10.0.1 Microsoft.Extensions.Configuration.Json, 10.0.1 System.Text.Json, 10.0.1

In [2]:
// Import the Microdoft Extensions AI NuGet Packages
#r "nuget: Microsoft.Extensions.AI, 10.1.1"
#r "nuget: Microsoft.Extensions.AI.Abstractions, 10.1.1"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 10.1.1-preview.1.25612.2"
// Import Azure & OpenAI NuGet Packages
#r "nuget: Azure.AI.OpenAI, 2.8.0-beta.1"
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: OpenAI, 2.8.0"

using Azure;
using Azure.AI.OpenAI;
using Microsoft.Extensions.AI;
using Microsoft.Extensions.Configuration;
using OpenAI.Chat;
using System.ClientModel;
using System.ComponentModel;
using System.Text.Json;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;
IChatClient chatClient;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");

    var apiKeyCredential = new ApiKeyCredential(azureOpenAIAPIKey);
    var azureOpenAIClient = new AzureOpenAIClient(new Uri(azureOpenAIEndpoint), apiKeyCredential);

    #pragma warning disable OPENAI001
    chatClient = azureOpenAIClient.GetChatClient(azureOpenAIModelDeploymentName).AsIChatClient();
}
else
{
    Console.WriteLine("Using OpenAI Service");

    var apiKeyCredential = new ApiKeyCredential(azureOpenAIAPIKey);
    var azureOpenAIClient = new AzureOpenAIClient(new Uri(azureOpenAIEndpoint), apiKeyCredential);

    #pragma warning disable OPENAI001
    chatClient = azureOpenAIClient.GetChatClient(azureOpenAIModelDeploymentName).AsIChatClient();
}

Installed Packages Azure.AI.OpenAI, 2.8.0-beta.1 Azure.Identity, 1.18.0-beta.2 Microsoft.Extensions.AI, 10.1.1 Microsoft.Extensions.AI.Abstractions, 10.1.1 Microsoft.Extensions.AI.OpenAI, 10.1.1-preview.1.25612.2 OpenAI, 2.8.0

Using Azure OpenAI Service


-----
### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great decision assistants, which can:
* Provide proper Decision Frames 
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the chat client instance is instantiated, it is ready to intake prompt instructions. In the prompt below, the chat client object is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [ ]:
using System.Text.Json;

// A simple Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios. 
""";

// Execute the prompt against the AI model
var simplePromptResponse = await chatClient.GetResponseAsync(simpleDecisionPrompt);
var responseString = simplePromptResponse.Messages[0].Contents[0].ToString();

// Display the response from the AI model
Console.WriteLine(responseString);

Here are **five decision-making frameworks** and how each can support better analysis and reasoning:  

---

**1. OODA Loop (Observe – Orient – Decide – Act)**  
- **Description:** Originally developed for military strategy, this iterative process involves observing the situation, orienting by analyzing the context, deciding on a course of action, and acting before repeating the cycle.  
- **Benefit:** It promotes rapid assessment and adjustment, helping decision-makers respond effectively in dynamic or uncertain environments such as crisis management or competitive markets.

---

**2. SWOT Analysis (Strengths – Weaknesses – Opportunities – Threats)**  
- **Description:** An analytical tool that categorizes internal strengths and weaknesses, and external opportunities and threats, to understand the strategic position.  
- **Benefit:** It encourages a balanced view that factors in both internal capabilities and external influences, helping to make more informed strategic or operational 

-----
### Step 3 - Execute a Decision Prompt with Streaming

Microsoft AI Extensions chat clients support response streaming when invoking the prompt. This allows responses to be streamed to the console as soon as they are made available by the LLM and service. Below the same decision prompt executed in Step 2 is used. However, notice that chunks are streamed and can be read by the user as soon as they are available. 

> 📝 **Note:** An average human can read between 25-40 Tokens / second. Therefore, while streaming certainly helps with providing AI output to the user, it begins to lose its effectiveness at large token velocity. Furthermore, while streaming certainly shows a responsive system, it does lose its effectiveness when the AI system needs to perform long processing. 

In [4]:
// Same Decision Intelligence prompt executed using Streaming output chunks 
await foreach (var streamChunk in chatClient.GetStreamingResponseAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Here are **five decision-making frameworks** and how they can improve analysis and reasoning:

---

**1. SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**  
- **Description:** A structured method for evaluating internal strengths and weaknesses, along with external opportunities and threats.  
- **How it helps:** By categorizing factors into these four quadrants, decision-makers get a balanced view of what can help or hinder success. It ensures that internal capabilities and external environmental factors are both considered, promoting more informed and strategic choices.

---

**2. Cost-Benefit Analysis (CBA)**  
- **Description:** A quantitative approach that compares the total expected costs of a decision to its anticipated benefits, often expressed in monetary terms.  
- **How it helps:** Provides a clear economic rationale for choices, helps prioritize options with the greatest net gain, and allows for transparent stakeholder communication regarding why a decision is

-----
### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can output Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting output. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [15]:
// A new decision prompt to help with describing decision-making frameworks using table Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

// Execute the prompt against the AI model
var simplePromptResponseWithMarkdownFormat = await chatClient.GetResponseAsync(simpleDecisionPromptWithMarkdownFormat);
var responseStringWithMarkdownFormat = simplePromptResponseWithMarkdownFormat.Messages[0].Contents[0].ToString();

// Display the response from the Semantic Kernel as Markdown
responseStringWithMarkdownFormat.DisplayAs("text/markdown");

| Decision-Making Framework | Description | How It Enhances Analysis and Reasoning |
|---------------------------|-------------|-----------------------------------------|
| SWOT Analysis | Identifies internal Strengths and Weaknesses, and external Opportunities and Threats. | Encourages a balanced view by assessing both internal capabilities and external factors, leading to more informed and strategic decisions. |
| Cost-Benefit Analysis (CBA) | Compares the projected costs and benefits of a decision. | Provides a quantitative basis for comparing alternatives, improving objectivity and minimizing emotional bias. |
| OODA Loop (Observe, Orient, Decide, Act) | Iterative cycle for rapid decision-making and adaptation. | Promotes continuous reassessment and adaptability, which is critical in fast-changing environments. |
| Decision Matrix | Scores and ranks options based on weighted criteria. | Helps prioritize alternatives according to defined, transparent criteria, reducing subjectivity in selection. |
| Six Thinking Hats | Uses six distinct thinking modes to assess a problem from multiple perspectives. | Encourages comprehensive exploration of facts, risks, creativity, and emotions, leading to well-rounded and less biased decisions. |

-----
### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

The Microsoft AI clients support the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different AI output experience. Non-reasoning models GPT-3.5 through GPT4.1 support settings like Temperature, LogProbs, TopK to optimize returns. Most recently (in the year 2026), most top performing Generative AI models are reasoning models, which simplify the configuration to reasoning effort (Minimal, Low, Medium, High) or token thinking budgets. This basically is a reasoning setting that correlates to how many resources the AI models spend "thinking" (internal monologue) before the AI executes the prompt instructions. 

> **📝 Note:** The supported prompt settings are dependent on the API plus the specific model version. For example, an AI model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available. Types of model execution (general versus reasoning) will also have different execution setting parameters. 

Execute the code cell below. Note there is a new ChatOptions object instantiated that specifies how we would like the AI to process the decision intelligence prompt. Note two key changes:
* **MaxOutputTokens** setting is set at 100, which will set a maximum of tokens allowed for the AI model to output
* **ResponseFormat** setting is set to JSON, which will return the response in a structured JSON object. An optional JSON schema can be supplied to enforce output in more specific ways.  

Notice the output difference below is greatly truncated, try changing MaxOutputTokens to a higher value and re-run. 

In [16]:
// Declare new chat options setting MaxOutputTokens and ResponseFormat to JSON explicitly
var chatOptions = new ChatOptions
{
    MaxOutputTokens = 100,
    ResponseFormat = Microsoft.Extensions.AI.ChatResponseFormat.Json
};

var simpleDecisionPromptWithJsonFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Return the response in JSON format. 
""";

// Execute the prompt against the AI model, pass in the chat options settings
var simplePromptResponse = await chatClient.GetResponseAsync(simpleDecisionPromptWithJsonFormat, chatOptions);
var responseString = simplePromptResponse.Messages[0].Contents[0].ToString();

// Display the response from the Semantic Kernel as Markdown
Console.WriteLine(responseString);


{
  "decision_making_frameworks": [
    {
      "name": "SWOT Analysis",
      "description": "SWOT (Strengths, Weaknesses, Opportunities, Threats) Analysis helps in evaluating internal and external factors influencing a decision. By organizing insights into these four categories, decision-makers can better assess trade-offs and strategic alignments.",
      "benefit": "Supports strategic thinking by balancing internal capabilities with external possibilities and risks."
    },
    {
      "name":


-----
### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decision Intelligence prompts, the typical best practices of prompt engineering apply. 

This includes: 
* Make the prompt more specific (i.e. decision intelligence)
* Add desired structure to the output with formatting
* Provide examples with few-shot prompting 
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  
* Cleanly dillineate sections in complex prompts 

A fundemental prompting best practice is to provide a common behavior across all the LLM interactions in a system prompt. The system prompt is passed in on every single call to the AI model. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common behavior across all your decision framework needs. This can be thought of as a **"persona"**. Furthermore, this common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.  


The core idea of a system prompt is to group all of common decision tasks, formats, decision approaches and place all of those things into the system prompt that can be re-used over and over for all of your prompt instructions & agentic subsequent calls. 

Execute the code cell below with a dynamic system prompt and prompt instructions. Notice the different behavior of the output for decision frameworks. Based on the new system prompt, the decision-making responses can be much more aligned with the provided decision intelligence information. 

In [ ]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

PromptExecutionSettings promptExecutionSettings = new();

if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
// Create a new KernelArguments object with the AzureOpenAI / OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(promptExecutionSettings);


var promptResponseWithTemplate = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt, kernelArguments);
var response = promptResponseWithTemplate.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

-----
### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full **Decision Intelligence** framework will not be used, rather a simple request for Artificial Intelligence to recommend a path forward (recommendation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [10]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommendation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

PromptExecutionSettings promptExecutionSettings = new();

if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
// Create a new KernelArguments object with the AzureOpenAI / OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(promptExecutionSettings);

var promptResponseScenario = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate, kernelArguments);
var response = promptResponseScenario.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### Recommendation

| Recommendation | Why (concise rationale) | Key trade-offs (most important) | Concrete next steps (first 12 months) | Contingencies / when to change course |
|---|---:|---|---|---|
| Start at the community college with an intentional, transfer-focused plan to a well-regarded four‑year university after 1–2 years. | Saves substantial money while giving Alex low‑risk space to explore majors (business, psychology, arts) and avoid costly major-switching at the expensive four‑year; preserve ability to transfer later to build the professional network and access internships once she’s more certain. | Pros: Big cost savings (reduces need for large loans); time to clarify major; lower financial risk.  Cons: Delay in full four‑year campus networking and on‑campus internship pipelines; transfer requires planning to avoid credit loss and timeline slip. | 1) Select a target 4‑year (or 2–3) now and obtain articulation/transfer agreement details. 2) Work with CC advisor to choose fully transferable general ed + exploratory courses aligned to possible majors. 3) Aim for a strong GPA (target 3.5+ for competitive transfers) and document coursework. 4) Pursue internships, volunteer, faculty contacts, and CC leadership roles to start building experience and references. 5) Apply for scholarships and external internships; keep family finances and loan needs updated. 6) Visit target campuses in year 1 and meet transfer admissions reps. | If by the end of year 1 Alex is clear about her major and can secure significant scholarships or other funding to offset the 4‑year cost, consider transferring early. If GPA or credits fall behind transfer requirements, extend CC plan to recover (summer courses, tutoring). If a target major is highly competitive, re-evaluate required GPA/portfolio and consider applying to multiple transfer schools or staying an extra semester to strengthen application. |

